This purpose of this project is to become more familiar with making a CNN in Tensorflow.  The aim is to differentiate between different forms of art and the data can be found at:  https://www.kaggle.com/thedownhill/art-images-drawings-painting-sculpture-engraving

As a note, there are a handful of corrupted imaged in the dataset that has to be removed before this will run.

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import matplotlib.pyplot as plt
mnist=tf.keras.datasets.mnist
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np



In [65]:
#shift data in various ways to make sure that the same image isn't seen repeatedly.  Also rescale the data.
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [167]:
img = load_img('dataset_updated/training_set/drawings/2051_mainfoto_05.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [168]:
#Preview what the image distortion does:
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [141]:
#Check our normalization factor
print(np.max(x))

255.0


In [142]:

'''
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), padding='same')) #Note: input_shape order had to be changed here
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy']) '''

In [199]:
model=Sequential()
model.add(Conv2D(64, (3,3), input_shape=(150, 150, 3), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

#model.add(Dense(1))
#model.add(Activation("sigmoid"))
model.add(Dense(6))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#model.fit(X, y, batch_size=32, validation_split=0.1, epochs=10

In [200]:
!ls dataset_updated

training_set   validation_set


In [201]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'dataset_updated/training_set',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        #class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'dataset_updated/validation_set',
        target_size=(150, 150),
        batch_size=batch_size,
        #class_mode='binary')
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels


Found 7721 images belonging to 6 classes.
Found 856 images belonging to 6 classes.


In [202]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5') 

Epoch 1/50
125/125 [==============================] - 215s 2s/step - loss: 1.1516 - acc: 0.5685 - val_loss: 0.8219 - val_acc: 0.6800
Epoch 2/50
 27/125 [=====>........................] - ETA: 2:25 - loss: 0.9254 - acc: 0.6227

/Users/ian/.pyenv/versions/3.6.5/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


125/125 [==============================] - 212s 2s/step - loss: 0.8988 - acc: 0.6605 - val_loss: 1.0310 - val_acc: 0.6048
Epoch 3/50
 29/125 [=====>........................] - ETA: 2:16 - loss: 0.8175 - acc: 0.7069

/Users/ian/.pyenv/versions/3.6.5/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6029312 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/Users/ian/.pyenv/versions/3.6.5/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1311848 bytes but only got 785. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/Users/ian/.pyenv/versions/3.6.5/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:764: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


125/125 [==============================] - 217s 2s/step - loss: 0.8054 - acc: 0.7100 - val_loss: 0.7414 - val_acc: 0.7222
Epoch 4/50
125/125 [==============================] - 206s 2s/step - loss: 0.7141 - acc: 0.7577 - val_loss: 0.6192 - val_acc: 0.7891
Epoch 5/50
124/125 [============================>.] - ETA: 1s - loss: 0.6675 - acc: 0.7611

KeyboardInterrupt: 

Stopped early, but after 5 epochs we have 75% accuracy.  There is still a lot of fine tuning to do with this model, such as trying different batch sizes, number of nodes per layer, and number of layers.